# Manual Activation Patching Experiments

### How activation patching works here (what goes where)

- **Source sentence (we capture from)**: `clean_text`
  - Set in each experiment (e.g., via `get_pattern_text(selected_pattern, CLEAN_TEXT_TYPE)` or `CUSTOM_CLEAN_TEXT`).
  - **Capture layer**: `capture_layer_idx` (resid_post at that layer). `-1` = last layer.
  - **Token selection**: `token_selection_strategy` chooses which token positions in `clean_text` provide activations (e.g., `KEYWORDS`, `MID_TOKENS`, `LAST_COUPLE`, `LAST_TOKEN`, `ALL_TOKENS`).

- **Target prompt (we patch into)**: depends on mode
  - **Zero-placeholder mode** (`zero_placeholder_mode=True`): use either a named `template_name` (with `0` markers) or a `prompt_input` string containing standalone `0` tokens. Those `0` positions are rendered as `<eos>` and become patch points.
  - **Placeholder-token mode** (`zero_placeholder_mode=False`): the notebook auto‑prepends `num_placeholder_tokens` copies of `<eos>` to `corrupted_text`; those initial tokens are the patch points.

- **Where we patch (layer)**: `patch_layer_idx` (resid_post). Defaults to `capture_layer_idx` if not provided. Can be an int or list of ints.

- **Generation**: We run with hooks so the model uses patched activations when computing next token logits and while generating continuation text.

- **Mental model**: Capture activations from `clean_text` at selected tokens → inject them at `<eos>` positions in the target prompt → generate continuation while hooks are active.


In [1]:
import sys
import os
import torch
import json
import random
from IPython.display import display, HTML

# Add TransformerLens to path
sys.path.append('/Users/ivanculo/Desktop/Projects/turn_point/third_party/TransformerLens')

# Import our activation patcher with all utilities
sys.path.append('/Users/ivanculo/Desktop/Projects/turn_point/manual_activation_patching')
from activation_patcher import ActivationPatcher, TokenSelectionStrategy
from interpretation_templates import INTERPRETATION_TEMPLATES

# Import utility functions from ActivationPatcher class
load_cognitive_patterns = ActivationPatcher.load_cognitive_patterns
get_pattern_by_index = ActivationPatcher.get_pattern_by_index
get_pattern_by_type = ActivationPatcher.get_pattern_by_type
get_random_pattern_by_type = ActivationPatcher.get_random_pattern_by_type
get_patterns_by_type = ActivationPatcher.get_patterns_by_type
list_available_pattern_types = ActivationPatcher.list_available_pattern_types
get_pattern_text = ActivationPatcher.get_pattern_text
get_template = ActivationPatcher.get_template
show_pattern_info = ActivationPatcher.show_pattern_info
clear_memory = ActivationPatcher.clear_memory
filter_patterns_by_count = ActivationPatcher.filter_patterns_by_count
get_filtered_patterns_by_type = ActivationPatcher.get_filtered_patterns_by_type

print("Imports and utilities loaded successfully!")

Imports and utilities loaded successfully!


## Initialize the Model and Load Dataset

In [2]:
# Choose your model here - change this and re-run to experiment with different models
# MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"  # Change to: gpt2-medium, EleutherAI/gpt-neo-125m, etc.
MODEL_NAME = "google/gemma-2-2b-it"
# # Set the BOS token for your model (change this based on your model)
BOS_TOKEN = "<bos>"  # Default for Gemma models

# # Initialize the activation patcher
patcher = ActivationPatcher(MODEL_NAME)

# Load the FULL cognitive patterns dataset (all 40 examples per pattern type)
patterns, pattern_types = load_cognitive_patterns()

# Show available pattern types
# list_available_pattern_types(pattern_types)

Supported models: ['gpt2-small', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'EleutherAI/gpt-j-6b', 'EleutherAI/gpt-neo-125m', 'EleutherAI/gpt-neo-1.3b', 'EleutherAI/gpt-neo-2.7b', 'facebook/opt-125m', 'facebook/opt-1.3b', 'facebook/opt-2.7b', 'facebook/opt-6.7b', 'EleutherAI/pythia-70m', 'EleutherAI/pythia-160m', 'EleutherAI/pythia-410m', 'EleutherAI/pythia-1b', 'EleutherAI/pythia-1.4b', 'EleutherAI/pythia-2.8b', 'google/gemma-2b', 'google/gemma-7b']
Using Apple Silicon GPU (MPS)
Loading model: google/gemma-2-2b-it on device: mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b-it into HookedTransformer
✓ Model loaded successfully
  - Family: unknown
  - Size: unknown
  - Layers: 26
  - Model dimension: 2304
  - Vocabulary size: 256000


## Experiment 1: Basic Activation Patching

Let's start with a simple experiment using one positive pattern:

### What goes where in Experiment 1

- **We capture from**: `clean_text` built from the dataset using `CLEAN_TEXT_TYPE`.
- **We patch into**: a prompt constructed from the `template_name` with `0` markers (zero‑placeholder mode). Each `0` is rendered as `<eos>` and marks a patch point in the prompt.
- **Capture layer**: `capture_layer_idx=-1` (last layer, resid_post).
- **Patch layer**: `patch_layer_idx=-1` (same last layer).
- **Tokens used to extract activations**: `TOKEN_STRATEGY` (default shown is `LAST_COUPLE` averaging the last few tokens).
- **Why**: This injects the end‑of‑thought representation from the clean example into the template to steer generation at the `<eos>` slots.


Available cognitive pattern types:
 1. Cognitive depletion pattern (40 examples)
 2. Intrusive suicidal fixation (40 examples)
 3. Negative self-evaluative loop (40 examples)
 4. Internal dialectical processing (40 examples)
 5. Fragmented perceptual reasoning (40 examples)
 6. Hyper-attuned interoception (40 examples)
 7. Autobiographical integration (40 examples)
 8. Over-elaborative recounting (40 examples)
 9. Entrapment cognition (40 examples)
10. Existential rumination (40 examples)
11. Learned helplessness loop (40 examples)
12. Instrumental suicidal reasoning (40 examples)
13. Cognitive disorganization (40 examples)
Available text types: positive, negative, transition

Available interpretation templates: ['cognitive_pattern', 'emotional_state', 'general_concept', 'decision_making']

💡 Each experiment can now filter to use 1-40 examples per pattern type at runtime!

In [11]:
# EXPERIMENT 1: Basic Activation Patching with Template (Zero-Placeholder Mode)

# ===== MODULAR CONFIGURATION SECTION =====
# Easily modify these parameters to experiment with different settings
NUM_EXAMPLES_PER_TYPE = 1  # Number of examples to use per pattern type (1-40)
PATTERN_TYPE = "Instrumental suicidal reasoning"  # Choose from the list above
PATTERN_INDEX_WITHIN_TYPE = 0 # If there are multiple examples of this type, choose which one (0-based)
CLEAN_TEXT_TYPE = "negative"  # "positive", "negative", or "transition"
TEMPLATE_NAME = "cognitive_pattern"  # Template to use for corrupted text
TOKEN_STRATEGY = TokenSelectionStrategy.LAST_TOKEN  # Change to: MID_TOKENS, LAST_COUPLE, LAST_TOKEN, ALL_TOKENS

# ===== ACTIVATION PATCHING PARAMETERS =====
OVERLAY_STRENGTH = 0  # 0.0 = no effect, 1.0 = full effect, >1.0 = amplified
REPLACING_MODE = 'normalized'  # 'normalized' (blend), 'addition' (add), 'direct' (replace)
NORMALIZE_MAGNITUDES = False  # Scale activations to match original magnitude
# ============================================

# Apply runtime filtering to use only NUM_EXAMPLES_PER_TYPE examples per pattern type
filtered_patterns, filtered_pattern_types = filter_patterns_by_count(pattern_types, NUM_EXAMPLES_PER_TYPE)

# Load the selected pattern and template from filtered dataset
selected_pattern = get_pattern_by_type(filtered_pattern_types, PATTERN_TYPE, PATTERN_INDEX_WITHIN_TYPE)
template = get_template(TEMPLATE_NAME)

# Get the clean text (what we want to capture activations from)
clean_text = get_pattern_text(selected_pattern, CLEAN_TEXT_TYPE)

# Run activation patching using zero-placeholder mode
print(f"\n⚡ Running activation patching (zero-placeholder mode)...")
print(f"  Clean text: {clean_text[:100]}...")

predicted_token, generated_text = patcher.patch_and_generate(
    clean_text=clean_text,
    corrupted_text=None,  # ignored in zero-placeholder mode when template_name is provided
    capture_layer_idx='all',  
    patch_layer_idx=3,    
    max_new_tokens=20,
    bos_token=BOS_TOKEN,
    token_selection_strategy=TOKEN_STRATEGY,
    num_strategy_tokens=5,
    zero_placeholder_mode=True,
    template_name=TEMPLATE_NAME,  # use template with '0' markers
    overlay_strength=OVERLAY_STRENGTH,  # Control activation intensity
    replacing_mode=REPLACING_MODE,  # Blending method
    normalize_magnitudes=NORMALIZE_MAGNITUDES  # Scale activations
)

print("\n" + "="*80)
print("🎊 EXPERIMENT 1 RESULTS:")
print("="*80)
print(f"Model: {patcher.model_name}")
print(f"Pattern Type: {PATTERN_TYPE}")
print(f"Pattern Name: {selected_pattern['cognitive_pattern_name']}")
print(f"Clean text type: {CLEAN_TEXT_TYPE}")
print(f"Template: {TEMPLATE_NAME}")
print(f"Token strategy: {TOKEN_STRATEGY.value}")
print(f"Overlay strength: {OVERLAY_STRENGTH}")
print(f"Replacing mode: {REPLACING_MODE}")
print(f"Normalize magnitudes: {NORMALIZE_MAGNITUDES}")
print(f"\n📊 Generated Text:")
print(f"{generated_text}")
print("="*80)


⚡ Running activation patching (zero-placeholder mode)...
  Clean text: I'm not being selfish; I just want to end this constant pain and suffering. It's exhausting trying t...
✅ Model hooks reset to clean state


  0%|          | 0/20 [00:00<?, ?it/s]


🎊 EXPERIMENT 1 RESULTS:
Model: google/gemma-2-2b-it
Pattern Type: Instrumental suicidal reasoning
Pattern Name: Suicidal Planning & Rationalization
Clean text type: negative
Template: cognitive_pattern
Token strategy: last_token
Overlay strength: 0
Replacing mode: normalized
Normalize magnitudes: False

📊 Generated Text:
<bos><start_of_turn>user
The following is a summary of the cognitive pattern:<end_of_turn>
<start_of_turn>model
nosWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOExceptionWithIOException


NameError: name 'reset_hooks' is not defined

### Manual zero‑placeholder prompt (what gets patched where)

- **We capture from**: the same `clean_text` as Experiment 1.
- **We patch into**: `manual_prompt` where each standalone `0` is rendered as `<eos>` and becomes a patch point.
- **Positions chosen**: The exact tokenized positions of the `<eos>` markers after rendering are located and used for patching.
- **Strategy/layers**: `TokenSelectionStrategy.LAST_COUPLE` from `capture_layer_idx=-1` → patched into `patch_layer_idx=-1`.
- **Tip**: Add or remove `0` markers to control how many `<eos>` positions get injected.


In [ ]:
# Manual prompt with '0' markers (zero-placeholder mode)
# Example: you can place '0' anywhere you want activations patched
manual_prompt = "0 0 0 I need to shift my perspective and find a constructive way to"

print(f"Manual prompt: {manual_prompt}")

predicted_token, generated_text = patcher.patch_and_generate(
    clean_text=clean_text,
    corrupted_text=None,  # ignored in zero-placeholder mode
    capture_layer_idx=-1,
    patch_layer_idx=-1,
    max_new_tokens=60,
    bos_token=BOS_TOKEN,
    token_selection_strategy=TokenSelectionStrategy.LAST_COUPLE,
    num_strategy_tokens=3,
    zero_placeholder_mode=True,
    prompt_input=manual_prompt
)

print("\n" + "="*100)
print("MANUAL PROMPT RESULTS:")
print("="*100)
print(f"Model: {patcher.model_name}")
print(f"BOS token used: '{BOS_TOKEN}'")
print(f"Clean text (first 100 chars): {clean_text[:100]}...")
print(f"\nGenerated Text:\n{generated_text}")
print("="*100)

#### 🔄 Reset Model State

Before running experiments, it's good practice to reset any lingering hooks from previous runs:

In [ ]:
# 🔄 RESET MODEL HOOKS - Run this cell to reset the model to clean state
# This is especially important when switching between different experiments

patcher.reset_hooks()

print("Model is now ready for clean experiments!")
print("Run this cell anytime you want to ensure no residual hooks are affecting your results.")

## Experiment 2: Different Layers Comparison

Let's see how patching at different layers affects the output:

### What goes where in Experiment 2 (layer sweep)

- **We capture from**: `clean_text` of `sample_pattern`.
- **We patch into**: `corrupted_text` with `num_placeholder_tokens=5` `<eos>` tokens prepended (non‑zero mode).
- **Capture/Patch layers**: same `layer_idx` per loop (tests early→late layers including `-1`).
- **Token selection**: `TokenSelectionStrategy.KEYWORDS` from the clean input.
- **Goal**: Compare how different patch layers influence continuation.


In [ ]:
# EXPERIMENT 2: Different Layers Comparison

# ===== MODULAR CONFIGURATION SECTION =====
NUM_EXAMPLES = 15  # Number of examples to use from dataset (1-40)
LAYERS_TO_TEST = [0, 3, 6, 9, -1]  # Early, middle, late, and final layers
TOKEN_STRATEGY = TokenSelectionStrategy.KEYWORDS  # Change to: MID_TOKENS, LAST_COUPLE, LAST_TOKEN, ALL_TOKENS
PATTERN_INDEX = 5  # Which pattern from full dataset to use (0-519)
CORRUPTED_TEXT = "I can't stop worrying about everything and feel completely"
MAX_NEW_TOKENS = 50
# ============================================

# Get limited patterns based on NUM_EXAMPLES
limited_patterns = patterns[:NUM_EXAMPLES * 13]  # 13 pattern types * NUM_EXAMPLES each
sample_pattern = patterns[PATTERN_INDEX] if PATTERN_INDEX < len(patterns) else patterns[5]

clean_text = sample_pattern['positive_thought_pattern']

print(f"📋 EXPERIMENT 2 SETUP - LAYER COMPARISON:")
print(f"📊 Using {NUM_EXAMPLES} examples per pattern type from dataset")
print(f"🧠 Pattern: {sample_pattern['cognitive_pattern_name']}")
print(f"🔧 Token strategy: {TOKEN_STRATEGY.value}")
print(f"📝 Corrupted prompt: {CORRUPTED_TEXT}")
print(f"🏗️ Testing layers: {LAYERS_TO_TEST}")
print("\n" + "="*100)

layer_results = {}

for layer_idx in LAYERS_TO_TEST:
    print(f"\n--- LAYER {layer_idx} ---")
    try:
        predicted_token, generated_text = patcher.patch_and_generate(
            clean_text=clean_text,
            corrupted_text=CORRUPTED_TEXT,
            num_placeholder_tokens=5,
            capture_layer_idx=layer_idx,  # Updated parameter name
            patch_layer_idx=layer_idx,    # Patch to same layer
            max_new_tokens=MAX_NEW_TOKENS,
            token_selection_strategy=TOKEN_STRATEGY,
            num_strategy_tokens=3
        )
        layer_results[layer_idx] = generated_text
        print(f"Generated: {generated_text}")
    except Exception as e:
        print(f"Error at layer {layer_idx}: {e}")
        layer_results[layer_idx] = f"Error: {e}"
    print("-" * 80)

In [ ]:
# EXPERIMENT 3: Multiple Patterns Comparison

# ===== MODULAR CONFIGURATION SECTION =====
NUM_EXAMPLES_PER_TYPE = 8  # Number of examples to use per pattern type (1-40)
NUM_PATTERNS_TO_TEST = 5  # How many different patterns to test
TOKEN_STRATEGY = TokenSelectionStrategy.KEYWORDS  # Change to: MID_TOKENS, LAST_COUPLE, LAST_TOKEN, ALL_TOKENS
CORRUPTED_TEXT = "I feel trapped and don't see a way forward because"
MAX_NEW_TOKENS = 65
CAPTURE_LAYER = -1
PATCH_LAYER = -1
NUM_PLACEHOLDERS = 5
# ============================================

# Apply runtime filtering to get limited dataset
filtered_patterns, filtered_pattern_types = filter_patterns_by_count(pattern_types, NUM_EXAMPLES_PER_TYPE)

# Get test patterns from filtered dataset
test_patterns = filtered_patterns[:NUM_PATTERNS_TO_TEST]

print(f"📋 EXPERIMENT 3 SETUP - MULTIPLE PATTERNS COMPARISON:")
print(f"📊 Using {NUM_EXAMPLES_PER_TYPE} examples per pattern type (runtime filtered)")
print(f"🔢 Total filtered patterns available: {len(filtered_patterns)} (was {len(patterns)})")
print(f"🧪 Testing {NUM_PATTERNS_TO_TEST} different patterns")
print(f"📝 Corrupted prompt: '{CORRUPTED_TEXT}'")
print(f"🔧 Token strategy: {TOKEN_STRATEGY.value}")
print(f"🏗️ Layers: capture={CAPTURE_LAYER}, patch={PATCH_LAYER}")
print(f"📍 Placeholders: {NUM_PLACEHOLDERS}, Max tokens: {MAX_NEW_TOKENS}")
print("\n" + "="*100)

pattern_results = []

for i, pattern in enumerate(test_patterns):
    print(f"\n--- PATTERN {i+1}/{NUM_PATTERNS_TO_TEST}: {pattern['cognitive_pattern_name']} ---")
    
    clean_text = pattern['positive_thought_pattern']
    
    print(f"Clean text (first 100 chars): {clean_text[:100]}...")
    
    try:
        predicted_token, generated_text = patcher.patch_and_generate(
            clean_text=clean_text,
            corrupted_text=CORRUPTED_TEXT,
            num_placeholder_tokens=NUM_PLACEHOLDERS,
            capture_layer_idx=CAPTURE_LAYER,  # Updated parameter name
            patch_layer_idx=PATCH_LAYER,      # Updated parameter name
            max_new_tokens=MAX_NEW_TOKENS,
            token_selection_strategy=TOKEN_STRATEGY,
            num_strategy_tokens=3
        )
        
        pattern_results.append({
            'pattern_name': pattern['cognitive_pattern_name'],
            'generated_text': generated_text
        })
        
        print(f"\n✨ GENERATED TEXT:")
        print(generated_text)
        
    except Exception as e:
        print(f"❌ Error with pattern {i+1}: {e}")
        pattern_results.append({
            'pattern_name': pattern['cognitive_pattern_name'],
            'generated_text': f"Error: {e}"
        })
    
    print("-" * 100)

print(f"\n🎊 EXPERIMENT 3 COMPLETED!")
print(f"✅ Successfully tested {len([r for r in pattern_results if not r['generated_text'].startswith('Error')])} out of {NUM_PATTERNS_TO_TEST} patterns")
print(f"📊 Using {NUM_EXAMPLES_PER_TYPE} examples per pattern type (runtime filtered)")

## Experiment 4: Different Number of Patch Positions

Let's experiment with varying the number of placeholder tokens we patch:

In [ ]:
# EXPERIMENT 4: Different Number of Patch Positions

# ===== MODULAR CONFIGURATION SECTION =====
NUM_EXAMPLES = 12  # Number of examples to choose from dataset (1-40)
PLACEHOLDER_COUNTS = [1, 3, 5, 7]  # Different numbers of placeholder tokens to test
TOKEN_STRATEGY = TokenSelectionStrategy.KEYWORDS  # Change to: MID_TOKENS, LAST_COUPLE, LAST_TOKEN, ALL_TOKENS
PATTERN_INDEX = 10  # Which pattern to use from the limited set
CORRUPTED_TEXT = "My mind keeps racing with negative thoughts and I feel"
MAX_NEW_TOKENS = 55
CAPTURE_LAYER = -1
PATCH_LAYER = -1
# ============================================

# Get limited patterns and select one for testing
limited_patterns = patterns[:NUM_EXAMPLES]
sample_pattern = limited_patterns[PATTERN_INDEX] if PATTERN_INDEX < len(limited_patterns) else limited_patterns[0]

clean_text = sample_pattern['positive_thought_pattern']

print(f"📋 EXPERIMENT 4 SETUP - PLACEHOLDER COUNT TESTING:")
print(f"📊 Selected from {NUM_EXAMPLES} examples in dataset")
print(f"🧠 Pattern: {sample_pattern['cognitive_pattern_name']}")
print(f"🔧 Token strategy: {TOKEN_STRATEGY.value}")
print(f"📝 Corrupted prompt: {CORRUPTED_TEXT}")
print(f"📍 Testing placeholder counts: {PLACEHOLDER_COUNTS}")
print(f"🏗️ Layers: capture={CAPTURE_LAYER}, patch={PATCH_LAYER}")
print("\n" + "="*100)

placeholder_results = {}

for num_placeholders in PLACEHOLDER_COUNTS:
    print(f"\n--- {num_placeholders} PLACEHOLDER TOKENS ---")
    try:
        predicted_token, generated_text = patcher.patch_and_generate(
            clean_text=clean_text,
            corrupted_text=CORRUPTED_TEXT,
            num_placeholder_tokens=num_placeholders,
            capture_layer_idx=CAPTURE_LAYER,  # Updated parameter name
            patch_layer_idx=PATCH_LAYER,      # Updated parameter name
            max_new_tokens=MAX_NEW_TOKENS,
            token_selection_strategy=TOKEN_STRATEGY,
            num_strategy_tokens=3
        )
        placeholder_results[num_placeholders] = generated_text
        print(f"✨ Generated: {generated_text}")
    except Exception as e:
        print(f"❌ Error with {num_placeholders} placeholders: {e}")
        placeholder_results[num_placeholders] = f"Error: {e}"
    print("-" * 80)

print(f"\n🎊 EXPERIMENT 4 COMPLETED!")
print(f"✅ Tested {len(PLACEHOLDER_COUNTS)} different placeholder configurations")
print(f"📊 Using pattern from limited dataset of {NUM_EXAMPLES} examples")

In [ ]:
# EXPERIMENT 6: Custom Pattern Testing - Interactive Sandbox

# ===== MODULAR CONFIGURATION SECTION =====
# You can change these to experiment with different combinations
NUM_EXAMPLES = 20  # Number of examples to choose from in dataset (1-40)

CUSTOM_CLEAN_TEXT = "I'm taking a moment to acknowledge my feelings and remind myself that challenges are temporary. I can take small, manageable steps forward and focus on what I can control right now."
CUSTOM_CORRUPTED_TEXT = "I don't know what to do anymore and feel completely lost"
CUSTOM_TOKEN_STRATEGY = TokenSelectionStrategy.KEYWORDS  # Change to: MID_TOKENS, LAST_COUPLE, LAST_TOKEN, ALL_TOKENS
CUSTOM_NUM_PLACEHOLDERS = 5
CUSTOM_CAPTURE_LAYER = -1  # Layer to capture activations from
CUSTOM_PATCH_LAYER = 3     # Layer to patch activations into
CUSTOM_MAX_TOKENS = 70

# Strategy comparison settings
COMPARISON_NUM_PLACEHOLDERS = 3
COMPARISON_MAX_TOKENS = 50
COMPARISON_STRATEGIES = [TokenSelectionStrategy.KEYWORDS, TokenSelectionStrategy.MID_TOKENS, 
                        TokenSelectionStrategy.LAST_COUPLE, TokenSelectionStrategy.LAST_TOKEN, 
                        TokenSelectionStrategy.ALL_TOKENS]
# ============================================

print("📋 EXPERIMENT 6 SETUP - CUSTOM PATTERN TESTING:")
print("="*100)
print(f"📊 Working with {NUM_EXAMPLES} examples from dataset")
print(f"📝 Clean text: {CUSTOM_CLEAN_TEXT}")
print(f"📝 Corrupted text: {CUSTOM_CORRUPTED_TEXT}")
print(f"🔧 Token strategy: {CUSTOM_TOKEN_STRATEGY.value}")
print(f"🏗️ Settings: {CUSTOM_NUM_PLACEHOLDERS} placeholders, capture layer {CUSTOM_CAPTURE_LAYER}, patch layer {CUSTOM_PATCH_LAYER}, {CUSTOM_MAX_TOKENS} max tokens")

try:
    predicted_token, generated_text = patcher.patch_and_generate(
        clean_text=CUSTOM_CLEAN_TEXT,
        corrupted_text=CUSTOM_CORRUPTED_TEXT,
        num_placeholder_tokens=CUSTOM_NUM_PLACEHOLDERS,
        capture_layer_idx=CUSTOM_CAPTURE_LAYER,  # Updated parameter name
        patch_layer_idx=CUSTOM_PATCH_LAYER,      # Updated parameter name
        max_new_tokens=CUSTOM_MAX_TOKENS,
        token_selection_strategy=CUSTOM_TOKEN_STRATEGY,
        num_strategy_tokens=3
    )
    
    print("\n✨ GENERATED TEXT:")
    print(generated_text)
    
except Exception as e:
    print(f"❌ Error in custom experiment: {e}")

print("\n" + "="*100)
print("🔧 TOKEN STRATEGY COMPARISON:")
print("="*100)
print(f"📊 Comparing strategies with {NUM_EXAMPLES} examples available")

# Compare all token strategies
for strategy in COMPARISON_STRATEGIES:
    print(f"\n--- STRATEGY: {strategy.value.upper()} ---")
    try:
        predicted_token, generated_text = patcher.patch_and_generate(
            clean_text=CUSTOM_CLEAN_TEXT,
            corrupted_text=CUSTOM_CORRUPTED_TEXT,
            num_placeholder_tokens=COMPARISON_NUM_PLACEHOLDERS,
            capture_layer_idx=-1,
            patch_layer_idx=-1,
            max_new_tokens=COMPARISON_MAX_TOKENS,
            token_selection_strategy=strategy,
            num_strategy_tokens=3
        )
        print(f"✨ Generated: {generated_text}")
    except Exception as e:
        print(f"❌ Error: {e}")
    print("-" * 60)

print(f"\n🎊 EXPERIMENT 6 COMPLETED!")
print(f"✅ Tested custom settings and {len(COMPARISON_STRATEGIES)} token strategies")
print(f"📊 Dataset scope: {NUM_EXAMPLES} examples per cognitive pattern type")

## 🛠️ Troubleshooting & Utilities

In [8]:
# 🛠️ TROUBLESHOOTING UTILITIES

# 1. Reset hooks if experiments behave unexpectedly
def quick_reset():
    patcher.reset_hooks()
    print("🔄 Hooks reset - Model is clean!")

# 2. Check model state - now uses the instance method
def check_model_info():
    patcher.check_model_info()

# 3. Clear memory if needed - now uses the static method
def clear_memory_util():
    ActivationPatcher.clear_memory()

# Quick access functions
print("Available utilities:")
print("- quick_reset() - Reset model hooks")
print("- check_model_info() - Display model details") 
print("- clear_memory_util() - Clear GPU/system memory")
print("- patcher.reset_hooks() - Direct reset call")

# Uncomment any line below to run:
# quick_reset()
# check_model_info()
# clear_memory_util()

Available utilities:
- quick_reset() - Reset model hooks
- check_model_info() - Display model details
- clear_memory_util() - Clear GPU/system memory
- patcher.reset_hooks() - Direct reset call


In [10]:
patcher.reset_hooks()

✅ Model hooks reset to clean state
